In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv('train_E6oV3lV.csv')
train.head()

,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [5]:
a = train.loc[0]['tweet']
a

' @user when a father is dysfunctional and is so selfish he drags his kids into his dysfunction.   #run'

In [6]:
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
ps = PorterStemmer()
a = re.sub('[^a-zA-Z]',' ',a)
a = a.lower()
a = a.split()
a = [ps.stem(word) for word in a if word not in set(stopwords.words('english'))]
a = ' '.join(a)  

In [7]:
type(a)

str

In [69]:
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
ps = PorterStemmer()
corpus=[]
for i in range(0,len(train)):
    tweet = re.sub('[^a-zA-Z]',' ',train['tweet'][i])
    tweet = tweet.lower()
    tweet = tweet.split()
    tweet = [ps.stem(word) for word in tweet if word not in set(stopwords.words('english'))]
    tweet = ' '.join(tweet)
    corpus.append(tweet)

In [85]:
len(corpus)

31962

In [86]:
y

0        0
1        0
2        0
3        0
4        0
        ..
31957    0
31958    0
31959    0
31960    1
31961    0
Name: label, Length: 31962, dtype: int64

In [88]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(corpus,train['label'],test_size=0.25,random_state=42)

In [89]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer()
X_train_vect = vectorizer.fit_transform(X_train)
X_test_vect = vectorizer.transform(X_test)

In [90]:
X_train_vect.shape

(23971, 26180)

In [91]:
from imblearn.over_sampling import SMOTE
x_resample, y_resample = SMOTE().fit_sample(X_train_vect, y_train)
# lets print the shape of x and y after resampling it
print(x_resample.shape)
print(y_resample.shape)

(44576, 26180)
(44576,)


In [92]:
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(n_estimators=500,num_leaves=50).fit(x_resample, y_resample)

In [93]:
x_test_resample, y_test_resample = SMOTE().fit_sample(X_test_vect, y_test)

In [94]:
from sklearn.metrics import accuracy_score, classification_report
y_pred = lgb.predict(x_test_resample)
accuracy_score(y_test_resample,y_pred)

0.9402583423035522

In [95]:
y_pred

array([0, 0, 0, ..., 1, 1, 1], dtype=int64)

In [96]:
df = pd.DataFrame()
df['pred'] = y_pred
df['actual'] = y_test_resample
df['pred'].value_counts()

0    8048
1    6816
Name: pred, dtype: int64

In [97]:
df['actual'].value_counts()

1    7432
0    7432
Name: actual, dtype: int64

In [98]:
cr = classification_report(y_test_resample, y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.91      0.98      0.94      7432
           1       0.98      0.90      0.94      7432

    accuracy                           0.94     14864
   macro avg       0.94      0.94      0.94     14864
weighted avg       0.94      0.94      0.94     14864



In [100]:
# Training model using Naive bayes classifier

from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()
mnb.fit(x_resample, y_resample)

MultinomialNB()

In [101]:
y_pred = mnb.predict(x_test_resample)
accuracy_score(y_test_resample,y_pred)

0.9146259418729817

In [102]:
cr = classification_report(y_test_resample, y_pred)
print(cr)

              precision    recall  f1-score   support

           0       0.91      0.92      0.91      7432
           1       0.92      0.91      0.91      7432

    accuracy                           0.91     14864
   macro avg       0.91      0.91      0.91     14864
weighted avg       0.91      0.91      0.91     14864

